# Runnables in LangChain

## What are Runnables?

**Runnables** are LangChain's fundamental building blocks for creating complex, composable workflows. They provide a unified interface for executing chains, allowing you to combine, branch, and parallelize operations with simple syntax.

### Why Use Runnables?

1. **Composability**: Chain operations together with `|` pipe operator
2. **Flexibility**: Mix sequential, parallel, and conditional flows
3. **Standardization**: Consistent interface across all components
4. **Control**: Fine-grained control over data flow and logic
5. **Reusability**: Build modular components that work together
6. **Scalability**: Easy to extend with custom logic

### Core Runnable Types

| Type | Purpose | Use Case |
|------|---------|----------|
| **RunnableSequence** | Execute steps in order | Linear workflows |
| **RunnableParallel** | Run independent tasks simultaneously | Multi-aspect analysis |
| **RunnableBranch** | Choose paths based on conditions | Dynamic routing |
| **RunnablePassthrough** | Pass data through without modification | Data flow control |
| **RunnableLambda** | Wrap Python functions as runnables | Custom logic |

### The Runnable Workflow

```
Input
  ↓
[Process] (Sequential/Parallel/Conditional)
  ↓
Intermediate Output
  ↓
[Process]
  ↓
Final Output
```

### Key Principle

**Runnables enable powerful composition through simple syntax**. The `|` operator lets you build complex workflows that remain readable and maintainable.

## Sequential Runnables

### Definition

A **RunnableSequence** executes multiple steps in a **strict linear order**. Each step completes before the next begins, with output automatically feeding into the next step's input.

### Key Features

- **Ordered Execution**: Steps run one after another
- **Auto Data Passing**: Output → Input (no manual passing)
- **LCEL Compatible**: Works with pipe operator `|`
- **Chaining**: Easily combine prompts, models, parsers
- **Linear Flow**: Perfect for sequential workflows

### Use Cases

- Step-by-step content generation
- Progressive transformations
- Report generation workflows
- Multi-stage processing pipelines
- Data refinement (raw → structured)

### When to Use

✅ **Use RunnableSequence when:**
- Steps depend on previous output
- Order matters
- Simple linear flow
- Transforming data step-by-step

❌ **Don't use when:**
- Steps are independent (use Parallel)
- Need conditional branching (use Branch)
- Complex custom logic needed


In [ ]:
# Sequential runnable

from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

prompt1 = PromptTemplate(
    template='Give some intresting facts about {topic}',
    input_variables=['topic']
)

chain = RunnableSequence(prompt1, model, parser)
result = chain.invoke({'topic': 'AI'})
print(result)

## Parallel Runnables

### Definition

A **RunnableParallel** executes multiple **independent chains simultaneously** based on the same input. All branches run at once (if system allows) and results are combined into a dictionary.

### Key Features

- **Simultaneous Execution**: All branches run in parallel
- **Independent Processing**: Each branch doesn't depend on others
- **Dictionary Output**: Results keyed by branch names
- **Efficient**: Faster than sequential for independent tasks
- **Composable**: Can combine with sequential and conditional

### How It Works

```
Input: {topic: 'AI'}
    ↓
┌───┴──────────────────┐
│  RunnableParallel    │
├───┬──────┬──────┬────┤
│   │      │      │    │
v   v      v      v    v
[Branch 1] [Branch 2] [Branch 3]
    ↓          ↓           ↓
Result 1   Result 2    Result 3
    └─────────┬─────────┘
         ↓
    {'branch1': ..., 'branch2': ..., 'branch3': ...}
```

### Use Cases

- Generate multiple perspectives simultaneously
- Create comprehensive analysis (summary + notes + facts)
- Multi-model comparisons
- Parallel content creation
- Extract different information from same input
- Content pipeline (different formats)

### Advantages

| Aspect | Sequential | Parallel |
|--------|-----------|----------|
| **Speed** | Slower | Faster |
| **Independence** | Dependent steps | Independent steps |
| **Complexity** | Simple | Medium |
| **Output** | Single value | Dictionary |

### When to Use

✅ **Use RunnableParallel when:**
- Branches are independent
- Need multiple outputs from one input
- Branches run on different resources
- Speed improvement needed

❌ **Don't use when:**
- Branches depend on each other
- Single output needed
- Sequential processing required


In [ ]:
# Parallel Runnable

from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableParallel
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

# Artical research
prompt1 = PromptTemplate(
    template='Generate 250 words summary on {topic}',
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template='Generate Bullet-point-notes on {topic}. cover only important points and make it concise.',
    input_variables=['topic']
)
prompt3 = PromptTemplate(
    template='Generate key-facts on {topic}',
    input_variables=['topic']
)
prompt4 = PromptTemplate(
    template='tell some possible applications on {topic}',
    input_variables=['topic']
)
prompt5 = PromptTemplate(
    template='Generate a Detailed Report from the content: \nSummary: {summary} \nPoint-Notes: {notes} \nKey-Facts: {facts} \nApplication: {application}',
    input_variables=['summary', 'notes', 'facts', 'application']
)

parallel_chain = RunnableParallel({
    'summary': RunnableSequence(prompt1, model, parser),
    'notes': RunnableSequence(prompt2, model, parser),
    'facts': RunnableSequence(prompt3, model, parser),
    'application': RunnableSequence(prompt4, model, parser)
})

chain = RunnableSequence(parallel_chain, prompt5, model, parser)
print(chain.invoke({'topic':'Agentic AI'}))



## Custom Data Flow and Logic

### RunnablePassthrough

**Definition**: Passes data through to later steps without modification, or adds new keys while preserving existing data.

**Key Features**:
- **Data Preservation**: Keeps original data intact
- **No Modification**: Doesn't transform input
- **Adding Keys**: Can add new data to existing
- **Branching**: Enables parallel access to original input
- **Flexibility**: Works with other runnables

**Use Cases**:
- Keep original input for later steps
- Branch to use same input multiple times
- Add metadata or computed fields
- Enable parallel branches from same data

**Example**:
```python
# Keep original while creating branches
RunnableParallel({
    'original': RunnablePassthrough(),
    'processed': chain1,
    'analyzed': chain2
})
# All get access to original input
```

### RunnableLambda

**Definition**: Wraps regular Python functions so they work as runnables in chains, enabling custom logic.

**Key Features**:
- **Function Wrapping**: Converts Python functions to runnables
- **Custom Logic**: Use any Python code in chains
- **Data Transformation**: Filter, format, compute, etc.
- **Simple Interface**: `RunnableLambda(lambda x: function(x))`
- **Composable**: Works with pipes and other runnables

**Use Cases**:
- Count words or characters
- Format/filter data
- Perform calculations
- Custom validation
- Data transformation
- Adding computed fields

**Example**:
```python
# Count words
word_counter = RunnableLambda(lambda x: len(x.split()))

# Extract specific field
extractor = RunnableLambda(lambda x: x['data']['value'])

# Custom transformation
formatter = RunnableLambda(lambda x: x.upper().strip())
```

### Comparison: Passthrough vs Lambda

| Feature | Passthrough | Lambda |
|---------|-------------|--------|
| **Modifies Data** | No | Yes |
| **Purpose** | Pass through | Custom logic |
| **Use** | Data preservation | Transformation |
| **Complexity** | None | Custom |

### When to Use

✅ **Use Passthrough when:**
- Need original input in branches
- Avoiding data loss
- Parallel access to input

✅ **Use Lambda when:**
- Custom data transformation needed
- Computation required
- Validation or filtering needed


In [ ]:
# runnable Passthrough and Lambda
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

prompt1 = PromptTemplate(
    template='Explain {topic} in siple words.',
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template='Give some key-points about {topic}',
    input_variables=['topic']
)
prompt3 = PromptTemplate(
    template='Generate some relivent question-answers about {topic}',
    input_variables=['topic']
)
chain = RunnableSequence(prompt1, model, parser)
parallel_chain = RunnableParallel({
    "points": RunnableSequence(prompt2, model, parser),
    'quiz': RunnableSequence(prompt3, model, parser ),
    'words': RunnableLambda(lambda x : len(x.split()))
})
pipeline = RunnableSequence(chain, parallel_chain)
result = pipeline.invoke({'topic': 'AI'})
print(result.keys())

print()
print(chain.invoke({'topic': 'ai'}))

## Conditional Logic (RunnableBranch)

### Definition

A **RunnableBranch** enables **conditional execution** - choosing different processing paths based on input data. It implements "if-then-else" logic for dynamic workflows.

### Key Features

- **Decision Making**: Evaluate conditions on data
- **Multiple Paths**: Different runnables for different conditions
- **Dynamic Routing**: Route inputs to appropriate handlers
- **Fallback**: Default path if no conditions match
- **Composable**: Works with other runnables

### How It Works

```
Input
  ↓
[Evaluate Condition 1] → True → Execute Path A → Output A
  ↓ False
[Evaluate Condition 2] → True → Execute Path B → Output B
  ↓ False
[Evaluate Condition 3] → True → Execute Path C → Output C
  ↓ False
[Execute Default Path] → Output Default
```

### Syntax

```python
RunnableBranch(
    (lambda x: condition1(x), runnable_if_true),
    (lambda x: condition2(x), runnable_if_true),
    default_runnable  # fallback if no conditions match
)
```

### Use Cases

- Content length → Different processors (summarize long, keep short)
- User role → Different information levels
- Input type → Different parsers
- Sentiment → Different response templates
- Language → Different language processors
- Query complexity → Different search strategies
- Data quality → Different validation chains

### Condition Examples

```python
# Length-based routing
lambda x: len(x.split()) > 500

# Attribute-based routing
lambda x: x.get('type') == 'email'

# Boolean flag routing
lambda x: x.get('is_premium', False)

# Range-based routing
lambda x: 18 <= x.get('age', 0) < 65

# Complex condition
lambda x: x.get('category') == 'urgent' and len(x.get('text', '')) > 1000
```

### When to Use

✅ **Use RunnableBranch when:**
- Need conditional execution
- Different paths for different inputs
- Dynamic workflow routing
- Multiple handlers for different cases

❌ **Don't use when:**
- Only one path needed
- All branches always execute (use Parallel)
- No conditions to evaluate


In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

prompt1 = PromptTemplate(
    template='Explain {topic} in siple words.',
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template='Summarize this text in unter 500 words.\n Text: {text}',
    input_variables=['text']
)

chain1 = prompt1 | model | parser 
chain2 = RunnableBranch(
    (lambda x : len(x.split()) > 500, chain1 | prompt2 | model | parser),
    RunnablePassthrough()
)

chain3 = chain1 | chain2
result = chain3.invoke({'topic': 'AI'})

print(result)

## Best Practices for Runnables

### 1. Choose the Right Runnable Type

**Decision Tree:**

```
Need multiple branches?
├─ Branches independent? 
│  ├─ YES → RunnableParallel
│  └─ NO → Sequential or Branch
├─ Need conditional routing?
│  ├─ YES → RunnableBranch
│  └─ NO → RunnableSequence
├─ Need to preserve input?
│  ├─ YES → RunnablePassthrough
│  └─ NO → Use pipe |
└─ Need custom logic?
   ├─ YES → RunnableLambda
   └─ NO → Use component
```

### 2. Build Complex Workflows Systematically

```python
# ✅ GOOD: Build and test step-by-step
step1 = prompt1 | model | parser
step2 = prompt2 | model | parser

# Test step1
result1 = step1.invoke({'input': 'test'})
assert result1  # Verify before combining

# Test step2
result2 = step2.invoke({'input': 'test'})
assert result2

# Now combine
sequential = step1 | step2  # Safe - both tested

# ❌ BAD: Build everything at once without testing
chain = (
    prompt1 | model | parser | 
    prompt2 | model | parser |
    prompt3 | model | parser
)  # Hard to debug if it fails
```

### 3. Handle Parallel Outputs Correctly

```python
# ✅ GOOD: Use dictionary keys to access parallel outputs
parallel = RunnableParallel({
    'summary': chain1,
    'facts': chain2
})

next_chain = RunnableLambda(
    lambda x: f"Summary: {x['summary']}\nFacts: {x['facts']}"
)

combined = parallel | next_chain

# ❌ BAD: Forget keys in next step
next_chain = lambda x: x + " more text"  # x is a dict, not string!
```

### 4. Optimize Parallel Execution

```python
# ✅ GOOD: Independent branches in parallel
parallel = RunnableParallel({
    'summary': summarize_chain,    # Use different models
    'entities': entity_chain,      # Independent analysis
    'sentiment': sentiment_chain   # No dependencies
})

# ❌ BAD: Dependencies between parallel branches
parallel = RunnableParallel({
    'step1': chain1,
    'step2': RunnableLambda(lambda x: process(x['step1']))  # Depends on step1!
})
# Should be sequential, not parallel
```

### 5. Error Handling and Validation

```python
from langchain_core.exceptions import LangChainException

def safe_invoke(runnable, inputs):
    try:
        return runnable.invoke(inputs)
    except LangChainException as e:
        print(f"Runnable error: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

# Test before production
result = safe_invoke(chain, {'input': 'test'})
```

### 6. Performance Considerations

```python
# ✅ GOOD: Use parallel for truly independent tasks
RunnableParallel({
    'analysis1': expensive_chain1,  # Different models
    'analysis2': expensive_chain2   # Run simultaneously
})  # Much faster!

# ✅ GOOD: Cache intermediate results
def cached_chain():
    intermediate = step1 | step2
    # Reuse intermediate multiple times
    return RunnableParallel({
        'branch1': intermediate | step3a,
        'branch2': intermediate | step3b
    })

# ❌ BAD: Sequential when parallel possible
step1 | step2a | step2b | step3  # Slow - sequential
# Better as: step1 | (step2a | step2b in parallel) | step3
```

### 7. Composition Patterns

```python
# Pattern 1: Simple sequence
chain = prompt | model | parser

# Pattern 2: Sequential with multiple steps
chain = prompt1 | model | parser | prompt2 | model | parser

# Pattern 3: Parallel analysis
chain = RunnableParallel({
    'summary': prompt1 | model | parser,
    'facts': prompt2 | model | parser,
    'keywords': prompt3 | model | parser
}) | merge_prompt | model | parser

# Pattern 4: Conditional routing
classifier = prompt_classify | model | parser_classify
routes = RunnableBranch(
    (lambda x: x.get('type') == 'long', long_handler),
    (lambda x: x.get('type') == 'short', short_handler),
    default_handler
)
chain = classifier | routes

# Pattern 5: Complex workflow
analysis = RunnableParallel({...})
conditional = RunnableBranch(...)
chain = input_processor | analysis | conditional | formatter
```

## Summary: Runnables

### Runnable Types Quick Reference

| Type | Use | Output | Speed |
|------|-----|--------|-------|
| **Sequence** | Linear flow | Single | Slowest |
| **Parallel** | Independent tasks | Dictionary | Fastest |
| **Branch** | Conditional routing | Route-specific | Medium |
| **Passthrough** | Data preservation | Unchanged | Instant |
| **Lambda** | Custom logic | Transformed | Depends |

### Key Takeaways

1. **Composition is Powerful**: `|` operator makes complex flows readable
2. **Sequential**: Output → Input (automatic data passing)
3. **Parallel**: Speed gains for independent operations
4. **Conditional**: Route based on data properties
5. **Custom Logic**: RunnableLambda for transformations
6. **Test Incrementally**: Build and test components separately
7. **Dictionary Keys**: Access parallel outputs by key names
8. **Error Handling**: Implement try-except for production

### Decision Matrix

| Need | Runnable Type |
|------|---------------|
| Linear workflow | Sequence (`\|`) |
| Multiple branches | Parallel |
| If-then-else logic | Branch |
| Keep original data | Passthrough |
| Custom function | Lambda |
| Combine all above | Nested composition |

### Performance Tips

✅ **DO:**
- Use Parallel for independent tasks
- Test components individually
- Cache expensive chains
- Handle errors gracefully
- Monitor chain execution

❌ **DON'T:**
- Force sequential when parallel possible
- Skip validation and testing
- Create circular dependencies
- Ignore error handling
- Build massive chains without tests

### Common Patterns

```python
# Pattern: Research article generation
parallel = RunnableParallel({
    'summary': summary_chain,
    'outline': outline_chain,
    'details': details_chain
})

merge = RunnableLambda(
    lambda x: f"Summary: {x['summary']}\n\n" +
              f"Outline: {x['outline']}\n\n" +
              f"Details: {x['details']}"
)

format_chain = merge | format_prompt | model | parser

result = parallel | format_chain
```

### Next Steps

After mastering runnables:
1. **Memory**: Add conversation history
2. **Agents**: Build autonomous decision systems
3. **RAG**: Implement retrieval-augmented generation
4. **Custom Runnables**: Create specialized components
5. **Production**: Deploy with monitoring and logging

### Additional Resources

- [LangChain Runnable Docs](https://python.langchain.com/docs/expression_language/runnables/)
- [LCEL Specification](https://python.langchain.com/docs/expression_language/)
- [Runnable API Reference](https://api.python.langchain.com/en/latest/runnables/)
- [LangChain Examples](https://python.langchain.com/docs/expression_language/cookbook/)
